# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import urllib

import json

# Import API key
from api_keys import geoapify_key

import os

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,colonia rincon viejo,17.0713,-95.1282,32.46,63,55,2.50,MX,1726428574
1,1,whitehorse,62.6669,-140.8534,9.62,76,100,1.99,CA,1726428576
2,2,buon ma thuot,27.5031,6.8259,36.45,22,33,4.76,DZ,1726428580
3,3,port hedland,12.7777,108.3078,20.41,98,99,2.37,VN,1726428582
4,4,northam,56.0243,8.8741,13.97,86,100,2.43,DK,1726428586


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', #Longtitute and Latitude as the x and y coordinates
    geo=True, #Enable geographic plotting
    tiles='OSM', #Background map tiles
    size='Humidity', #Point size proportional to humidity
    color='City',  # Color by the categorical column 'City'
    frame_width=700, #Map width
    frame_height=500, #Map height
    hover_cols=['City', 'Country', 'Humidity'] #Show additional info on hover
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
#Converted from kelvin to C in WeatherPy.ipynb
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 15) &
    (city_data_df['Max Temp'] <= 30) &
    (city_data_df['Humidity'] <= 75) &
    (city_data_df['Wind Speed'] < 7)
]

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,nampula,40.9228,87.0750,21.32,24,55,4.29,CN,1726428615
17,17,hasaki,-14.1465,38.9953,23.63,58,0,3.33,MZ,1726428619
20,20,guangyuan,40.7330,-5.2398,17.84,20,0,5.02,ES,1726428630
33,33,blackmans bay,-9.4631,-37.4179,24.74,66,29,6.97,BR,1726428678
37,37,nejo,59.7127,-115.5193,17.35,42,69,0.72,CA,1726428696


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Pandas copy function to create DataFrame hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
15,nampula,CN,40.9228,87.0750,24,
17,hasaki,MZ,-14.1465,38.9953,58,
20,guangyuan,ES,40.7330,-5.2398,20,
33,blackmans bay,BR,-9.4631,-37.4179,66,
37,nejo,CA,59.7127,-115.5193,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel

# Base URL for the Geoapify Places API (for searching places like hotels)
base_url = "https://api.geoapify.com/v2/places"

# Set the search radius (in meters)
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame for the current city
    lat = row['Lat']
    lng = row['Lng']

    # Update the params dictionary with the current city coordinates
    params = {
        "categories": "accommodation.hotel",  # Search for hotels
        "filter": f"circle:{lng},{lat},{radius}",  # Use circle filter with lat/lng and radius
        "bias": f"proximity:{lng},{lat}",  # Proximity bias
        "limit": 1,  # Get only the nearest hotel
        "apiKey": geoapify_key
    }

    full_url = base_url + "?" + urllib.parse.urlencode(params)


    # Make the API request
    response = requests.get(full_url)

    # print(f"{row=} {full_url=} {response.content.decode()}")

    # Check if the request was successful
    if response.status_code == 200:
        try:
            # Convert the API response to JSON format
            name_address = response.json()

            # Extract the hotel name from the response if available
            if name_address['features']:
                hotel_name = name_address['features'][0]['properties']['name']
                # Update the 'Hotel Name' column in the DataFrame
                hotel_df.at[index, 'Hotel Name'] = hotel_name
            else:
                # If no hotel is found, set 'No Hotel Found'
                hotel_df.at[index, 'Hotel Name'] = "No Hotel Found"
        except (KeyError, IndexError) as e:
            print(f"Error parsing API response: {e}")
            hotel_df.at[index, 'Hotel Name'] = "No Hotel Found"
    else:
        print(f"Error: {response.status_code} for city {row['City']}: {response.content.decode()}")
        hotel_df.at[index, 'Hotel Name'] = "No Hotel Found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display the updated DataFrame
hotel_df.head()


Starting hotel search
nampula - nearest hotel: No Hotel Found
hasaki - nearest hotel: No Hotel Found
guangyuan - nearest hotel: No Hotel Found
blackmans bay - nearest hotel: No Hotel Found
nejo - nearest hotel: No Hotel Found
port elizabeth - nearest hotel: No Hotel Found
almazan - nearest hotel: No Hotel Found
palatka - nearest hotel: No Hotel Found
tuscaloosa - nearest hotel: No Hotel Found
voi - nearest hotel: No Hotel Found
dapa - nearest hotel: No Hotel Found
pangody - nearest hotel: No Hotel Found
niebull - nearest hotel: No Hotel Found
Error parsing API response: 'name'
puerto san carlos - nearest hotel: No Hotel Found
as sulayyil - nearest hotel: No Hotel Found
dunhua - nearest hotel: No Hotel Found
zav'yalovo - nearest hotel: Vaalbos Chalets
sur - nearest hotel: No Hotel Found
vylkove - nearest hotel: No Hotel Found
comayagua - nearest hotel: No Hotel Found
talnakh - nearest hotel: No Hotel Found
zhanjiang - nearest hotel: No Hotel Found
arar - nearest hotel: No Hotel Found
us

,City,Country,Lat,Lng,Humidity,Hotel Name
15,nampula,CN,40.9228,87.0750,24,No Hotel Found
17,hasaki,MZ,-14.1465,38.9953,58,No Hotel Found
20,guangyuan,ES,40.7330,-5.2398,20,No Hotel Found
33,blackmans bay,BR,-9.4631,-37.4179,66,No Hotel Found
37,nejo,CA,59.7127,-115.5193,42,No Hotel Found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
#%%capture --no-display
# Create the hover message including City, Country, Hotel Name, and Humidity
hotel_df['Hover Info'] = hotel_df.apply(
    lambda row: f"Longitude: {row['Lng']}<br>Latitude: {row['Lat']}<br>City: {row['City']}<br>Humidity: {row['Humidity']}%<br>Hotel: {row['Hotel Name']}", axis=1)

# Plot the points on a map with the updated hover information
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,  # Activate geographic plot
    color='City',  # Color by the categorical column 'City'
    size=10,  # Size of the points
    hover_cols=['Hover Info'],  # Use the custom hover message
    #tiles='ESRI', 
    tiles='OSM',  # Base map tiles
    frame_width=800,  # Adjust width
    frame_height=600  # Adjust height
)

# Show the map with hover info
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hover Info)